### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Leo XIONG tlxiong

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class or office hours. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in HW04.ipynb and HW04.html to Canvas's HW04 assignment

  As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [3]:
import pandas as pd
from io import StringIO
from sklearn import svm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

## 1. Feature engineering (one-hot encoding and data imputation)

(Note: This paragraph is not instructions but rather is to communicate context for this exercise. We use the same Titanic data we used in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.


In [6]:
df = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv")
df = df[['Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]
print(df.head(7))

   Survived  Pclass     Sex   Age  SibSp  Parch
0         0       3    male  22.0      1      0
1         1       1  female  38.0      1      0
2         1       3  female  26.0      0      0
3         1       1  female  35.0      1      0
4         0       3    male  35.0      0      0
5         0       3    male   NaN      0      0
6         0       1    male  54.0      0      0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [8]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass','Sex','Age','SibSp','Parch']]

knn.fit(X, y)

ValueError: could not convert string to float: 'male'

In [18]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass','Sex','Age','SibSp','Parch']]

# knn.fit(X, y)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [20]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass', 'Age', 'SibSp', 'Parch']]

knn.fit(X, y)
X.isna().any()

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [23]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass', 'Age', 'SibSp', 'Parch']]

# knn.fit(X, y)
X.isna().any()

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [25]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass', 'SibSp', 'Parch']]

knn.fit(X, y)
y_pred = knn.predict(X)
accuracy = accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.6644219977553311


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [27]:
df = df.join(pd.get_dummies(df['Sex'], drop_first=True).rename(columns={'male': 'male'}))

knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass', 'SibSp', 'Parch', 'male']]

knn.fit(X, y)
y_pred = knn.predict(X)
accuracy = accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.7441077441077442


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [29]:
df['age'] = df['Age'].fillna(df['Age'].median())

knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y = df['Survived']
X = df[['Pclass', 'SibSp', 'Parch', 'male', 'age']]

knn.fit(X, y)
y_pred = knn.predict(X)
accuracy = accuracy_score(y, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8630751964085297


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [31]:
df = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv")
X = df[['cyl','disp','hp','drat','wt','qsec','vs','am','gear','carb']]
y = df['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [33]:
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)
y_pred = linear_model.predict(X_test)

y_train_pred = linear_model.predict(X_train)
y_test_pred = linear_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

coefficients = linear_model.coef_
intercept = linear_model.intercept_

print(f"For Linear Regression: \nMSEtrain: {mse_train} \nMSEtest: {mse_test} \nw coefficients: {coefficients} \nw intercept: {intercept} \n \n")




lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

coefficients = lasso_model.coef_
intercept = lasso_model.intercept_

print(f"For Lasso: \nMSEtrain: {mse_train} \nMSEtest: {mse_test} \nw coefficients: {coefficients} \nw intercept: {intercept} \n \n")




ridge_model = Ridge()
ridge_model.fit(X_train, y_train)

y_train_pred = ridge_model.predict(X_train)
y_test_pred = ridge_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

coefficients = ridge_model.coef_
intercept = ridge_model.intercept_

print(f"For Ridge: \nMSEtrain: {mse_train} \nMSEtest: {mse_test} \nw coefficients: {coefficients} \nw intercept: {intercept} \n \n")

For Linear Regression: 
MSEtrain: 0.3856870040556244 
MSEtest: 30.227426389844194 
w coefficients: [ 1.08241627  0.02500827  0.02593759  3.12997296 -7.34326296  3.93233873
 -4.08551838 -1.2164054   5.98718293 -0.77097967] 
w intercept: -70.29912808004312 
 

For Lasso: 
MSEtrain: 5.692738046538883 
MSEtest: 12.99192397981821 
w coefficients: [-0.         -0.03718773 -0.01681757  0.         -0.          0.
  0.          0.          0.         -0.84712891] 
w intercept: 33.80119680807491 
 

For Ridge: 
MSEtrain: 1.9856526768793952 
MSEtest: 11.198177807346077 
w coefficients: [-0.01392446 -0.00744628  0.00400324  0.79271685 -3.21854592  1.09825372
 -0.48236992  0.47079795  1.49546846 -1.04547895] 
w intercept: 7.249748161940916 
 



### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?

Training Data: <b>Linear regression</b>, because it has the lowest MSE, meaning it fits the data better

Test Data: <b>Ridge</b>, because it reduces over/under fitting on unknown data and reduces noise

Feature Selection: <b>Lasso</b>, because many of the w coefficients are close to 0, so lasso would already make those 0s irrelevant